In [1]:
import random  
import operator

class Triest_base():
    def __init__(self, M):
        self.S = set()
        self.t = 0
        self.tau = 0
        self.tau_count = {}
        self.M = M
        
    def flippedBiasedCoin(self):
        hot=random.choices(population = ["heads","tail"], weights=[self.M/self.t,1-(self.M/self.t)])
        return hot[0]          
            
            
    def sampleEdge(self,edge):
        if self.t<=self.M:
            return True
        elif self.flippedBiasedCoin()=="heads":
            random_edge = random.sample(self.S,1)[0]
            self.S.remove(random_edge)
            self.updateCounters(operator.sub, random_edge)
            return True
        else:
            return False

    def updateCounters(self,operator,edge):
        u=edge[0]
        v=edge[1]
        #u = edge.split("-")[0]
        #v = edge.split("-")[1]
        nu = set()
        nv = set()
        #Alla neighbours
        for elem in self.S:
            if u == elem[0]:
                nu.add(elem[1])
            if u == elem[1]:
                nu.add(elem[0])
            if v == elem[0]:
                nv.add(elem[1])
            if v == elem[1]:
                nv.add(elem[0])
        #Loopa igenom alla snitten av neighbours till u och v        
        #print(nu.intersection(nv))
        for c in nu.intersection(nv):
            self.tau = operator(self.tau,1)
            self.tau_count[c] = operator(int(self.tau_count.get(c) or 0),1)
            self.tau_count[u] = operator(int(self.tau_count.get(u) or 0),1)
            self.tau_count[v] = operator(int(self.tau_count.get(v) or 0),1)
    def start(self, dataset):
        for edge in dataset:
            self.t = self.t + 1
            if self.t % 500000 == 0:
                print("t:",self.t,"tau:",self.tau)
            if self.sampleEdge(edge):
                self.S.add(edge)
                self.updateCounters(operator.add,edge)
        
        eps = (self.t*(self.t-1)*(self.t-2))/(self.M*(self.M-1)*(self.M-2))
        if eps < 1:
            eps = 1
        return self.tau*eps


In [2]:
class Triest_improved():
    def __init__(self, M):
        self.S = set()
        self.t = 0
        self.tau = 0
        self.tau_count = {}
        self.M = M
        
    def flippedBiasedCoin(self):
        hot=random.choices(population = ["heads","tail"], weights=[self.M/self.t,1-(self.M/self.t)])
        return hot[0]          
            
            
    def sampleEdge(self,edge):
        if self.t<=self.M:
            return True
        elif self.flippedBiasedCoin()=="heads":
            random_edge = random.sample(self.S,1)[0]
            self.S.remove(random_edge)
            #self.updateCounters(operator.sub, random_edge)
            return True
        else:
            return False

    def updateCounters(self,operator,edge):
        u=edge[0]
        v=edge[1]
        #u = edge.split("-")[0]
        #v = edge.split("-")[1]
        nu = set()
        nv = set()
        #Alla neighbours
        for elem in self.S:
            if u == elem[0]:
                nu.add(elem[1])
            if u == elem[1]:
                nu.add(elem[0])
            if v == elem[0]:
                nv.add(elem[1])
            if v == elem[1]:
                nv.add(elem[0])
        #Loopa igenom alla snitten av neighbours till u och v        
        #print(nu.intersection(nv))
        n = (self.t-1)*(self.t-2)
        n_t = n/(self.M*(self.M-1))
        if n_t < 1:
            n_t = 1
        for c in nu.intersection(nv):
            self.tau = operator(self.tau,n_t)
            self.tau_count[c] = operator(int(self.tau_count.get(c) or 0),n_t)
            self.tau_count[u] = operator(int(self.tau_count.get(u) or 0),n_t)
            self.tau_count[v] = operator(int(self.tau_count.get(v) or 0),n_t)
    def start(self, dataset):
        for edge in dataset:
            self.t = self.t + 1
            if self.t % 500000 == 0:
                print("t:",self.t,"tau:",self.tau)
            self.updateCounters(operator.add,edge)
            if self.sampleEdge(edge):
                self.S.add(edge)  
        return self.tau


In [3]:
dataset = set()
with open("web-NotreDame.txt") as file:
    for line in file:
        if "#" in line:
            continue
        else:
            edge = tuple(line.replace("\n","").split("\t"))
            if int(edge[0]) > int(edge[1]):
                edge = tuple([int(edge[1]),int(edge[0])]) 
            else:
                edge = tuple([int(edge[0]),int(edge[1])])
            if edge in dataset or edge[0] == edge[1]:
                continue
            else:
                dataset.add(edge)



In [4]:
base = Triest_base(1000)
print("The number of triangles are:", base.start(dataset))
impr = Triest_improved(1000)
print("The number of triangles are:", impr.start(dataset))

t: 500000 tau: 0
t: 1000000 tau: 0
The number of triangles are: 0.0
t: 500000 tau: 684075.7626846847
t: 1000000 tau: 6089001.838738739
The number of triangles are: 7263016.118524524
